In [102]:
import numpy as np 
import pandas as pd
import nltk
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop_words_ = set(stopwords.words('english'))
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
wn = WordNetLemmatizer()

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/brendanhutchinson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brendanhutchinson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/brendanhutchinson/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [125]:
df = pd.read_csv('books_with_blurbs.csv')

In [126]:
df['text'] = df['Author'] + " " + df['Publisher'] + " " + df['Title'] + " " + df['Blurb']

In [127]:
def black_txt(token):
    return token not in stop_words_ and token not in list(string.punctuation) and len(token)>2 

def clean_text(corpus):
    cleaned_text = []
    cleaned_text2 = []
    corpus = re.sub("'", "", corpus)
    corpus = re.sub("(\\d|\\W)+"," ",corpus)
    corpus = re.sub("/[^A-Z0-9]+/ig", "",corpus)
    corpus = corpus.replace('_','')
    cleaned_text = [wn.lemmatize(word, pos = 'v') for word in word_tokenize(corpus.lower()) if black_txt(word)]
    cleaned_text2 = [word for word in cleaned_text if black_txt(word)]
    return " ".join(cleaned_text2)


In [128]:
train_data = df.drop(columns = ['ISBN'])
word_features = ['text']
train_data['text'] = train_data['text'].map(str).apply(clean_text)

In [129]:
cvec = CountVectorizer()
text = train_data['text']
cvec.fit(text)
vec_mat = cvec.transform(text)
vec_df = pd.DataFrame(cvec.transform(text).toarray(),
                     columns=[k for k, v in sorted(cvec.vocabulary_.items(), 
                     key=lambda item: item[1])])

In [142]:
book_search = input('what book do you want to find?')


what book do you want to find?dragons


In [143]:
# ibook_search = 'books about pigs and cheese'

search = clean_text(book_search)
search_vec = cvec.transform([search])

In [144]:
from sklearn.metrics.pairwise import cosine_similarity

test = cosine_similarity(search_vec, vec_mat)
#vec_df.apply(lambda x: cosine_similarity(x,search_vec))

In [145]:
best_book = np.argmax(test[0])

print('Title : {}'.format(train_data.loc[best_book, 'Title']))
print('Author : {}'.format(train_data.loc[best_book, 'Author']))
print('Publisher : {}'.format(train_data.loc[best_book, 'Publisher']))
print('Blurb : {}'.format(train_data.loc[best_book, 'Blurb']))


Title : Dragons: A Natural History
Author : Karl Shuker
Publisher : Simon &amp; Schuster
Blurb : For millennia, few animals have fascinated mankind more than dragons. But until now, no rigorous survey has captured them in all their glorious variety. "Dragons: A Natural History" is that survey...a one-of-a-kind book, bringing these captivating creatures to life with lavish illustrations and vivid commentary. This marvelous compendium will take you on a journey that begins with the earliest serpent dragons and continues to the present day. Along the way, Dr. Karl Shuker, one of the world's leading experts on dracontology, shares his wealth of knowledge on: ,Dragon dwellings: These magnificent beasts have been found in an astounding number of places. Dragons and their near relatives have found niches in every ecosystem on the planet -- from the mountains of Greece to the forests of northern Europe to the volcanic plain of Mesoamerica to the river valleys of China -- and have, as a consequ